### IMPORT DEPENDENCIES

In [ ]:
#Install all dependencies
!pip3 install torch torchvision

In [ ]:
!pip show gymnasium ale-py

In [ ]:
import gym
from stable_baselines3 import A2C
#Allows to speed up training by running it in multiple environments
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os

### TEST ENVIRONMENT

In [ ]:
# Inorder to use atari environments we need to install:
# http://www.atarimania.com/roms/Roms.rar

In [ ]:
#!python -m atari_py_import_roms .\Roms\ROMS

In [ ]:
environment_name = "ALE/Breakout-v4"

In [ ]:
env = gym.make(environment_name, render_mode = "human", obs_type = "rgb")

In [ ]:
env.reset()

In [ ]:
env.action_space

In [ ]:
env.observation_space
#Observation_space = box
#0 - 255: value range
#210 - height
#160 - 3
#3 - image based model

In [ ]:
#Testing the environment
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, truncated, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

### VECTORISE ENVIRONMENT AND TRAIN MODEL

In [ ]:
# 'make_atari_env' allows us to create wrapped atari environments
# 'n_envs' makes 4 environments
#' seed' - Reproducable reports
env = make_atari_env('ALE/Breakout-v5', n_envs = 4, seed = 0)

#'n_stacks' stacks the environmnets together
env = VecFrameStack(env, n_stack = 4)

In [ ]:
env.reset()

In [ ]:
env.render("human")

In [ ]:
#Setting up log path
log_path = os.path.join('Training', 'Logs')

#Uses Convoultional Neural Network rather than a multiperceptron
model = A2C('CnnPolicy', env, verbose = 1, tensorboard_log = log_path)

In [ ]:
#Training
model.learn(total_timesteps = 100000) 

### SAVE AND RELOAD MODEL

In [ ]:
a2c_path = os.path.join('Training', 'Saved Models', 'A2C Vyshnav')

In [ ]:
model.save(a2c_path)

In [ ]:
del model

In [ ]:
model = A2C.load(a2c_path, env)

### EVALUVATE AND TEST

In [ ]:
#Recreating env
env = make_atari_env('ALE/Breakout-v5', n_envs = 1, seed = 0)

#'n_stacks' stacks the environmnets together
env = VecFrameStack(env, n_stack = 4)

In [ ]:
evaluate_policy(model, env, n_eval_episodes = 10, render = True)

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render("human")
# model playing the game (breakout ball)